<a href="https://colab.research.google.com/github/kabilan942/Natural-Language-Processing/blob/main/Summarization_DailyMail/cnn_dailymail_rouge_bleu_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Legal Extractive summ - Legal PART 1

## Modules Required

In [ ]:
!pip install transformers
!pip install rouge-score
!pip install datasets
!pip install tensorflow
!pip install rouge
!pip install pyyaml h5py
!pip install pip install rouge-score
!pip install evaluate
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00
ERROR: Operation cancelled by user
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bf12cdefbfea60e6d0716dd58a94167f939ce468e7abb618f700f50d0ee5a3be
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.0 M

In [ ]:
import sys
sys.path.insert(0, '../')
import glob
import os
import json
import nltk
nltk.download('punkt')
from rouge_score import rouge_scorer
from sklearn.preprocessing import MinMaxScaler
from rouge import Rouge
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
from evaluate import load

from transformers import BertTokenizer, TFBertModel
from tensorflow import keras

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Getting the data

In [ ]:
train=pd.read_csv("/content/training.csv")

In [ ]:
train = train[5:10]

((5, 3), pandas.core.frame.DataFrame)

In [ ]:
train.dropna(inplace=True)

In [ ]:
train_source = train['article'].values.tolist()
train_summary = train['highlights'].values.tolist()
data_source = train_source
data_summary = train_summary

## Calculating Saliency score for each sentence with its summary

In [ ]:
def saliency_score_rougef(doc, summary, alpha=0.5):
  # takes in two inputs and returns the saliency score
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
  scores = scorer.score(doc, summary)
  R1 = scores['rouge1'][2]
  R2 = scores['rouge2'][2]

  return alpha * R1 + (1 - alpha) * R2

def saliency_score_rouger(doc, summary, alpha=0.5):
  # takes in two inputs and returns the saliency score
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
  scores = scorer.score(doc, summary)
  R1 = scores['rouge1'][0]
  R2 = scores['rouge2'][0]

  return alpha * R1 + (1 - alpha) * R2

def calculate_bleu_score(reference, candidate):
    # Convert the reference and candidate sentences into lists of tokens
    reference_tokens = nltk.word_tokenize(reference)
    candidate_tokens = nltk.word_tokenize(candidate)

    # Calculate BLEU score using NLTK's sentence_bleu function
    smoothing_function = nltk.translate.bleu_score.SmoothingFunction().method1
    bleu_score = nltk.translate.bleu_score.sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothing_function)

    return bleu_score

def saliency_score_bleu(doc, summary):
  # inputs: (doc[reference_sentence], summary[candidate_sentence])
  # output: bleu score

  return calculate_bleu_score(doc, summary)

bertscore = load("bertscore")

def saliency_score_bertscore(doc, summary):
  # takes in two inputs and returns the saliency score
  scores = bertscore.compute(predictions=[doc], references=[summary], model_type="distilbert-base-uncased")  #

  return scores['f1'][0]

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)   # returns list of sentences from para
    return sents

In [ ]:
input_data = [[],[],[],[],[],[],[],[],[]]  # [docid, sentid, sentences, length of sentence, length of summary, saliency score ROUGE f1, saliency score ROUGE Recall, saliency score BLEU]
total_sentences_doc = []
total_sentences_summary = []
for i in tqdm(range(len(data_source))):
  splitted_doc = split_to_sentences(data_source[i])
  total_sentences_doc.append(len(splitted_doc))
  doc_sents = []
  for ind in range(len(splitted_doc)):
    if len(splitted_doc[ind].split())>5:
      doc_sents.append(splitted_doc[ind])
  summary = data_summary[i]
  total_sentences_summary.append(len(split_to_sentences(summary)))
  saliency_list_rougef = []
  saliency_list_rouger = []
  saliency_list_bleu = []
  saliency_list_bertscore = []
  for sent in doc_sents:
    saliency_list_rougef.append(saliency_score_rougef(sent, summary))
    saliency_list_rouger.append(saliency_score_rouger(sent, summary))
    saliency_list_bleu.append(saliency_score_bleu(sent, summary))
    saliency_list_bertscore.append(saliency_score_bertscore(sent, summary))
  saliency_list_rougef = np.array(saliency_list_rougef)
  saliency_list_rouger = np.array(saliency_list_rouger)
  saliency_list_bleu = np.array(saliency_list_bleu)
  saliency_list_bertscore = np.array(saliency_list_bertscore)
  mms_rougef = MinMaxScaler()
  mms_rouger = MinMaxScaler()
  mms_bleu = MinMaxScaler()
  mms_bertscore = MinMaxScaler()
  if len(saliency_list_rougef)>0:
    scaled_saliency_list_rougef = mms_rougef.fit_transform(saliency_list_rougef.reshape(-1,1))
  if len(saliency_list_rouger)>0:
    scaled_saliency_list_rouger = mms_rouger.fit_transform(saliency_list_rouger.reshape(-1,1))
  if len(saliency_list_bleu)>0:
    scaled_saliency_list_bleu = mms_bleu.fit_transform(saliency_list_bleu.reshape(-1,1))
  if len(saliency_list_bertscore)>0:
    scaled_saliency_list_bertscore = mms_bertscore.fit_transform(saliency_list_bertscore.reshape(-1,1))
  for j in range(len(doc_sents)):
    len_sent = len(doc_sents[j].split())
    input_data[0].append(i)
    input_data[1].append(j)
    input_data[2].append(doc_sents[j])
    input_data[3].append(len_sent)
    input_data[4].append(len(split_to_sentences(summary)))
    input_data[5].append(scaled_saliency_list_rougef[j][0])
    input_data[6].append(scaled_saliency_list_rouger[j][0])
    input_data[7].append(scaled_saliency_list_bleu[j][0])
    input_data[8].append(scaled_saliency_list_bertscore[j][0])

100%|██████████| 5/5 [03:38<00:00, 43.62s/it]


In [ ]:
avg_sent_per_doc = sum(total_sentences_doc)/len(data_source)
avg_sent_per_sum = sum(total_sentences_summary)/len(data_summary)
longest_sent = max(total_sentences_doc)

In [ ]:
print('Average sentences per doc: ',avg_sent_per_doc)
print('Average sentences per summary: ',avg_sent_per_sum)
print('Longest sentence in doc:', longest_sent)

Average sentences per doc:  42.2
Average sentences per summary:  3.4
Longest sentence in doc: 65


In [ ]:
df = pd.DataFrame({'doc_id':input_data[0],'sent_id':input_data[1],'Sentence':input_data[2],'Sentence Length':input_data[3], 'Summary Length':input_data[4],
                   'Saliency Score ROUGE f1':input_data[5],'Saliency Score ROUGE Recall':input_data[6], 'Saliency Score BLEU':input_data[7], 'Saliency Score BERTScore':input_data[8]})
df.head()

,doc_id,sent_id,Sentence,Sentence Length,Summary Length,Saliency Score ROUGE f1,Saliency Score ROUGE Recall,Saliency Score BLEU,Saliency Score BERTScore
0,0,0,He's been accused of making many a fashion fau...,13,3,0.227589,0.212766,0.019982,0.148855
1,0,1,But the Prime Minister seems to be deaf to his...,11,3,0.404407,0.356383,0.104890,0.200535
2,0,2,Yesterday David Cameron was seen in the same p...,18,3,1.000000,1.000000,0.481209,1.000000
3,0,3,"Mr Cameron, who is in Lanzarote with his famil...",27,3,0.853806,1.000000,0.199887,0.995623
4,0,4,Retread: David Cameron with Samantha yesterday.,6,3,0.000000,0.000000,0.000000,0.042387


In [ ]:
#df.to_parquet('saliency_sentences1.parquet', engine='fastparquet')
df.to_csv('saliency_sentences_dm_rougefr_bleu_bert_1.csv')

## Generate word embeddings for each sentence in doc

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # 'bert-base-cased' used initially
enc_model = TFBertModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

### Removing sentences with more than `sent_limit` tokens and storing in df - saliency_sentences_sent_limit

In [ ]:
column_names = ['Sentence','Saliency Score ROUGE f1','Saliency Score ROUGE Recall','Saliency Score BLEU','Saliency Score BERTScore']

In [ ]:
saliency_sentences = pd.read_csv('saliency_sentences_dm_rougefr_bleu_bert_1.csv',  usecols=column_names)

saliency_sentences.head()

,Sentence,Saliency Score ROUGE f1,Saliency Score ROUGE Recall,Saliency Score BLEU,Saliency Score BERTScore
0,"14:11 EST, 25 October 2013 .",0.000000,0.000000,0.000000,0.031703
1,"15:36 EST, 25 October 2013 .",0.000000,0.000000,0.000000,0.000000
2,The bishop of the Fargo Catholic Diocese in No...,0.787966,0.851276,0.866267,0.913845
3,The state Health Department has issued an advi...,0.173988,0.146504,0.002594,0.600494
4,Bishop John Folda (pictured) of the Fargo Cath...,1.000000,1.000000,1.000000,1.000000


In [ ]:
input_data = saliency_sentences.to_numpy().transpose()

sentences = input_data[0]

In [ ]:
input_data.shape

(5, 108)

In [ ]:
sent_limit = 50

In [ ]:
max_len = 0
count_sent_limit=0
summ=0
out_range=[]
for i in tqdm(range(len(sentences))):
  encoded_text = tokenizer(sentences[i], return_tensors='tf')
  l = len(np.array(encoded_text['input_ids'])[0])
  summ+=l
  if l>sent_limit:
    count_sent_limit+=1
    out_range.append(i)
    continue
  if l>max_len:
    max_len=l

avg_token_size = summ/len(sentences)
print()
print('Maximum length of token after removing sentences with tokens greater than sent_limit: ',max_len)
print('Average number of tokens per sentence:', avg_token_size)
print('Total number of sentences with tokens greater than 50:', count_sent_limit)

100%|██████████| 108/108 [00:00<00:00, 873.05it/s]


Maximum length of token after removing sentences with tokens greater than sent_limit:  48
Average number of tokens per sentence: 31.24074074074074
Total number of sentences with tokens greater than 50: 12


In [ ]:
input_data_sent_limit = np.delete(input_data,out_range, axis=1)   # add in prev loop with an if statement that if l>100 -> delete that row

input_data_sent_limit.shape

(5, 96)

In [ ]:
saliency_sentences_sent_limit = pd.DataFrame(data = input_data_sent_limit.transpose(), columns=column_names)

saliency_sentences_sent_limit.head()

,Sentence,Saliency Score ROUGE f1,Saliency Score ROUGE Recall,Saliency Score BLEU,Saliency Score BERTScore
0,"14:11 EST, 25 October 2013 .",0.0,0.0,0.0,0.031703
1,"15:36 EST, 25 October 2013 .",0.0,0.0,0.0,0.0
2,The bishop of the Fargo Catholic Diocese in No...,0.787966,0.851276,0.866267,0.913845
3,The state Health Department has issued an advi...,0.173988,0.146504,0.002594,0.600494
4,Bishop John Folda (pictured) of the Fargo Cath...,1.0,1.0,1.0,1.0


In [ ]:
saliency_sentences_sent_limit.to_csv('saliency_sentences_limit50_dm_rougefr_bleu_bert_1.csv')